# 安裝相關套件([talib](http://mrjbq7.github.io/ta-lib/), [pandas_datareader](https://pandas-datareader.readthedocs.io/en/latest/))並 import

In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install TA-Lib pandas-datareader

import datetime

from talib.abstract import *
import pandas
import pandas_datareader

(Reading database ... 144702 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


# 查看 talib 中技術指標的文件

In [ ]:
print(MA)

MA([input_arrays], [timeperiod=30], [matype=0])

Moving average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
    matype: 0 (Simple Moving Average)
Outputs:
    real


In [ ]:
print(MACD)

MACD([input_arrays], [fastperiod=12], [slowperiod=26], [signalperiod=9])

Moving Average Convergence/Divergence (Momentum Indicators)

Inputs:
    price: (any ndarray)
Parameters:
    fastperiod: 12
    slowperiod: 26
    signalperiod: 9
Outputs:
    macd
    macdsignal
    macdhist


# 修改此處設定可以抓取並計算不同的資料

functions 為一個 list of tuples，要增加技術指標只要增加一列即可

tuple 中有三項：指標名，產出 column rename 的後綴名，指標的參數

其中參數為一個 dict，請參閱上面的文件來下

In [ ]:
source = 'yahoo'
symbol = '^TWII'
start = datetime.date(2010, 1, 1)
end = datetime.date(2020, 8, 31)
functions = [
  ('MA', '', {}),
  ('MACD', '10_30', {'fastperiod': 10, 'slowperiod': 30}),
]

In [ ]:
org_data = pandas_datareader.DataReader(symbol, data_source=source, start=start, end=end).rename(columns=lambda x:x.lower())
org_data

,high,low,open,close,volume,adj close
Date,,,,,,
2010-01-04,8240.450195,8143.450195,8222.419922,8207.849609,6429400.0,8207.817383
2010-01-05,8283.639648,8162.540039,8277.709961,8211.400391,6762000.0,8211.368164
2010-01-06,8327.870117,8216.919922,8237.099609,8327.620117,6421600.0,8327.587891
2010-01-07,8369.549805,8233.589844,8344.559570,8237.419922,7133000.0,8237.387695
2010-01-08,8290.660156,8178.930176,8266.870117,8280.900391,5764400.0,8280.868164
...,...,...,...,...,...,...
2020-08-26,12833.290039,12734.690430,12768.959961,12833.290039,3413200.0,12833.290039
2020-08-27,12960.679688,12786.450195,12888.650391,12797.309570,4956500.0,12797.309570
2020-08-28,12799.160156,12674.570312,12759.299805,12728.849609,3774200.0,12728.849609


In [ ]:
data = [org_data]

for func, name, param in functions:
    index = Function(func.lower())(org_data, **param)
    if index.ndim == 1:
        index.name = func.lower() + name
    else:
        index.rename(columns=lambda x:x + name, inplace=True)
    data.append(index)

data = pandas.concat(data, axis=1)
data

,high,low,open,close,volume,adj close,ma,macd10_30,macdsignal10_30,macdhist10_30
Date,,,,,,,,,,
2010-01-04,8240.450195,8143.450195,8222.419922,8207.849609,6429400.0,8207.817383,NaN,NaN,NaN,NaN
2010-01-05,8283.639648,8162.540039,8277.709961,8211.400391,6762000.0,8211.368164,NaN,NaN,NaN,NaN
2010-01-06,8327.870117,8216.919922,8237.099609,8327.620117,6421600.0,8327.587891,NaN,NaN,NaN,NaN
2010-01-07,8369.549805,8233.589844,8344.559570,8237.419922,7133000.0,8237.387695,NaN,NaN,NaN,NaN
2010-01-08,8290.660156,8178.930176,8266.870117,8280.900391,5764400.0,8280.868164,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-08-26,12833.290039,12734.690430,12768.959961,12833.290039,3413200.0,12833.290039,12623.074935,163.918439,214.565636,-50.647197
2020-08-27,12960.679688,12786.450195,12888.650391,12797.309570,4956500.0,12797.309570,12644.393913,161.968755,204.046260,-42.077504
2020-08-28,12799.160156,12674.570312,12759.299805,12728.849609,3774200.0,12728.849609,12662.636914,150.546055,193.346219,-42.800164
